<h1 style="color: #492c68;">01 | BASICS</h1>

<h2 style="color: #327a81;">Libraries</h2>

In [1]:
## Basic libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

## Settings

pd.set_option('display.max_columns', None) # display all columns
import warnings
warnings.filterwarnings('ignore') # ignore warnings

<h2 style="color: #327a81;">Data Read</h2>

In [2]:
data= pd.read_csv("Netflx_mood_analysis.csv")

In [3]:
data.head(1)

,show_id,type,title,director,cast,country,date_added,rating,listed_in,n_seasons,movie_lenght,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Not Provided,United States,2021-09-25,PG-13,Documentaries,NaN,90.0,"As her father nears the end of his life, filmm..."


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       8807 non-null   object 
 1   type          8807 non-null   object 
 2   title         8807 non-null   object 
 3   director      8807 non-null   object 
 4   cast          8807 non-null   object 
 5   country       8807 non-null   object 
 6   date_added    8709 non-null   object 
 7   rating        8807 non-null   object 
 8   listed_in     8807 non-null   object 
 9   n_seasons     2676 non-null   float64
 10  movie_lenght  6128 non-null   float64
 11  description   8807 non-null   object 
dtypes: float64(2), object(10)
memory usage: 825.8+ KB


In [5]:
df = data.copy()

<h1 style="color: #492c68;">01 | MOOD ANALYSIS</h1>

- We will use the pretrained model "Emotion English DistilRoBERTa-base" from Hugging Face, a fine-tuned version of RoBERTa (famous NLP model that works searching for relation between language and context) 
- This pipeline will classify the sypnopsis from the catalogue in emotion. 